In [ ]:
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.1 MB/s eta 0:00:00


In [ ]:
## for data
import pandas as pd
import numpy as np
import re
from datetime import datetime
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for machine learning
from sklearn import metrics, preprocessing
## for deep learning
from tensorflow.keras import models, layers, utils  #(2.6.0)


from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import RegexpTokenizer
import re
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)
stem = StemmerFactory().create_stemmer()
stopword = StopWordRemoverFactory().create_stop_word_remover()

In [ ]:
user = pd.read_csv('tourism_rating.csv')
tourism = pd.read_csv('tourism_with_id.csv')
print(tourism.info())
print(user.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 437 entries, 0 to 436
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Place_Id      437 non-null    int64  
 1   Place_Name    437 non-null    object 
 2   Description   437 non-null    object 
 3   Category      437 non-null    object 
 4   City          437 non-null    object 
 5   Price         437 non-null    int64  
 6   Rating        437 non-null    float64
 7   Time_Minutes  205 non-null    float64
 8   Coordinate    437 non-null    object 
 9   Lat           437 non-null    float64
 10  Long          437 non-null    float64
 11  Unnamed: 11   0 non-null      float64
 12  Unnamed: 12   437 non-null    int64  
dtypes: float64(5), int64(3), object(5)
memory usage: 44.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype
---  ------         -------------

In [ ]:
#Drop Column
tourism.drop(['Time_Minutes','Coordinate','Lat','Long','Unnamed: 11','Unnamed: 12'],axis=1,inplace=True)

# Detect Missing Value
print("==========  Missing Values ==========")
print(user.isna().sum())
print(tourism.isna().sum())

# handlimg missing value
user["User_Id"] = user["User_Id"].fillna(0)
user["Place_Id"] = user["Place_Id"].fillna(0)
user["Place_Ratings"] = user["Place_Ratings"].fillna(0)

print(user.isna().sum())

print(tourism.info())
print(user.info())

==========  Missing Values ==========
User_Id          0
Place_Id         0
Place_Ratings    0
dtype: int64
Place_Id       0
Place_Name     0
Description    0
Category       0
City           0
Price          0
Rating         0
dtype: int64
User_Id          0
Place_Id         0
Place_Ratings    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 437 entries, 0 to 436
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Place_Id     437 non-null    int64  
 1   Place_Name   437 non-null    object 
 2   Description  437 non-null    object 
 3   Category     437 non-null    object 
 4   City         437 non-null    object 
 5   Price        437 non-null    int64  
 6   Rating       437 non-null    float64
dtypes: float64(1), int64(2), object(4)
memory usage: 24.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column         Non-Null Coun

In [ ]:
tags = [i.split("|") for i in tourism["Description"].unique()]
columns = list(set([i for lst in tags for i in lst]))
for col in columns:
    tourism[col] = tourism["Description"].apply(lambda x: 1 if col in x else 0)

<ipython-input-19-41622cb22b45>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tourism[col] = tourism["Description"].apply(lambda x: 1 if col in x else 0)
<ipython-input-19-41622cb22b45>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tourism[col] = tourism["Description"].apply(lambda x: 1 if col in x else 0)
<ipython-input-19-41622cb22b45>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once u

In [ ]:
# fig, ax = plt.subplots(figsize=(20,5))
# sns.heatmap(tourism==0, vmin=0, vmax=1, cbar=False, ax=ax).set_title("Products x Features")
# plt.show()

In [ ]:
tmp = user.copy()
user = tmp.pivot_table(index="User_Id", columns="Place_Id", values="Place_Ratings")
missing_cols = list(set(tourism.index) - set(user.columns))
for col in missing_cols:
    user[col] = np.nan
user = user[sorted(user.columns)]

In [ ]:
user = pd.DataFrame(preprocessing.MinMaxScaler(feature_range=(0.5,1)).fit_transform(user.values), 
columns=user.columns, index=user.index)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:473: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:474: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [ ]:
split = int(0.8*user.shape[1])
dtf_train = user.loc[:, :split-1]
dtf_test = user.loc[:, split:]

In [ ]:
# Select a user
i = 1
train = dtf_train.iloc[i].to_frame(name="y")
test = dtf_test.iloc[i].to_frame(name="y")
# add all the test products but hide the y
tmp = test.copy()
tmp["y"] = np.nan
train = train.append(tmp)
train = train.tail(train.shape[0]-1)

<ipython-input-22-b54c0f272d2e>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train.append(tmp)


In [ ]:
def Cleaning(data):
    data = data.lower()
    data = stem.stem(data)
    data = stopword.remove(data)
    html_pattern = re.compile('<.*?>')
    data = html_pattern.sub(r'', data)
    tokenizer = RegexpTokenizer(r'\w+')
    data = tokenizer.tokenize(data)
    data = " ".join(data)
    return data

In [ ]:
# tourism.Description = tourism.Description.apply(Cleaning)
# tourism

In [ ]:
# shapes
usr = train[["y"]].fillna(0).values.T
prd = tourism.drop(["Place_Name","Category",'Description','City','Price','Rating'],axis=1).values
print("Users", usr.shape, " x  Tourism", prd.shape)

Users (1, 437)  x  Tourism (437, 438)


In [ ]:
# # usr_ft(users,fatures) = usr(users,products) x prd(products,features)
# usr = usr.astype(int)
# usr_ft = np.dot(usr, prd)
# # normalize
# weights = usr_ft / usr_ft.sum()
# # predicted rating(users,products) = weights(users,fatures) x prd.T(features,products)
# pred = np.dot(weights, prd.T)
# test = test.merge(pd.DataFrame(pred[0], columns=["yhat"]), how="left", left_index=True, right_index=True).reset_index()
# test = test[~test["y"].isna()]
# pred

In [ ]:
import tensorflow as tf
# usr_ft(users,fatures) = usr(users,products) x prd(products,features)
usr_ft = tf.matmul(usr, prd)
# normalize
weights = usr_ft / tf.reduce_sum(usr_ft, axis=1, keepdims=True)
# rating(users,products) = weights(users,fatures) x prd.T(features,products)
pred = tf.matmul(weights, prd.T)

In [ ]:
test = test.merge(pd.DataFrame(pred[0], columns=["yhat"]), how="left", left_index=True, right_index=True).reset_index()
test = test[~test["y"].isna()]
test

,Place_Id,y,yhat
17,367,2.0,365.434402
21,371,3.0,369.406515
34,384,5.0,382.315883
39,389,2.0,387.281024
40,390,2.0,388.274052
57,407,5.0,405.155533
63,413,3.0,411.113703
76,426,4.0,424.023070
87,437,4.5,NaN


In [ ]:
def mean_reciprocal_rank(y_test, predicted):
    score = []
    for product in y_test:
        if product in predicted:
            mrr = 1 / (list(predicted).index(product) + 1)
            score.append(mrr)
        else:
            score.append(0)
    return np.mean(score)

In [ ]:
print("--- user", i, "---")
top = 5
y_test = test.sort_values("y", ascending=False)["Place_Id"].values[:top]
print("y_test:", y_test)
predicted = test.sort_values("yhat", ascending=False)["Place_Id"].values[:top]
print("predicted:", predicted)
true_positive = len(list(set(y_test) & set(predicted)))
print("true positive:", true_positive, "("+str(round(true_positive/top*100,1))+"%)")
print("accuracy:", str(round(metrics.accuracy_score(y_test,predicted)*100,1))+"%")
print("mrr:", mean_reciprocal_rank(y_test, predicted))

--- user 1 ---
y_test: [384 407 437 426 371]
predicted: [426 413 407 390 389]
true positive: 2 (40.0%)
accuracy: 0.0%
mrr: 0.26666666666666666


In [ ]:
# See predictions details
test.merge(
       tourism[["Place_Name","Category",'Description','City','Price','Rating']], left_on="Place_Id", 
       right_index=True
).sort_values("yhat", ascending=False)

,Place_Id,y,yhat,Place_Name,Category,Description,City,Price,Rating
76,426,4.0,424.023070,Patung Buddha Empat Rupa,Budaya,"Terletak di sudut Pantai Ria Kenjeran, Patung ...",Surabaya,0,4.3
63,413,3.0,411.113703,Jembatan Merah,Budaya,Jembatan Merah merupakan salah satu monumen se...,Surabaya,0,4.5
57,407,5.0,405.155533,Kebun Binatang Surabaya,Cagar Alam,Kebun Binatang Surabaya (KBS) (Dialek Arekan: ...,Surabaya,15000,4.3
40,390,2.0,388.274052,Flower Farm Setiya Aji,Cagar Alam,Di taman bunga Setiya Aji Flower Farm kita bis...,Semarang,7500,4.1
39,389,2.0,387.281024,Durian Gardens and Tourism Education Watu Simbar,Cagar Alam,"Perlu diketahui, watu simbar ini merupakan lok...",Semarang,5000,4.1
34,384,5.0,382.315883,Kampung Batik Gedong Semarang,Budaya,Kampung Batik Semarang merupakan salah satu ka...,Semarang,0,4.4
21,371,3.0,369.406515,Gua Maria Kerep Ambarawa,Cagar Alam,"Gua Maria Kerep (Gua Maria Kerep Ambarawa, dis...",Semarang,2000,4.8
17,367,2.0,365.434402,Kampoeng Rawa,Cagar Alam,Kampoeng Rawa (disebut juga Kampung Rawa) adal...,Semarang,3000,4.5


In [ ]:
import pickle

# Simpan model
with open('content_based_model_tf.pkl', 'wb') as file:
    pickle.dump(pred, file)

In [ ]:
import pickle
with open('content_based_model_tf.pkl', 'rb') as file:
    objek = pickle.load(file)
    print(objek)

tf.Tensor(
[[  0.99553815   1.9887511    2.9815451    3.97475805   4.96797101
    5.96118396   6.95435502   7.94760987   8.9392308    9.93244376
   10.92565671  11.91886966  12.91208262  13.90529557  14.89819431
   15.89373246  16.88694541  17.88015836  18.87295236  19.86658427
   20.85979723  21.85106205  22.844275    23.83943609  24.83032385
   25.82353681  26.818656    27.81186895  28.80317567  29.79829486
   30.79146592  31.78513972  32.77793372  33.76924044  34.76245339
   35.75757258  36.75078554  37.74441745  38.7376304   39.73084335
   40.72405631  41.71726926  42.71006326  43.70369517  44.69648917
   45.6881101   46.68291508  47.67654699  48.66975994  49.66297289
   50.65576689  51.6493988   52.64261176  53.63540576  54.62708953
   55.62023964  56.61504462  57.60825757  58.60188948  59.59510243
   60.58831539  61.58110939  62.5747413   63.56795425  64.5611672
   65.55438016  66.54713226  67.53848087  68.53401902  69.52723197
   70.51998408  71.51323893  72.50687084  73.5000837